# Call me! Crash course de llamado de variantes genómicas usando datos de sequenciación masiva

## Cómo filtrar archivos VCF, python style

Al igual que hemos hecho los demás días, lo primero es preparar el escenario, vamos a emplear las bibliotecas [Pandas](https://pandas.pydata.org/) y [Seaborn](https://seaborn.pydata.org/) para explorar nuestros archivos VCF.

Para instalar pandas y seaborn en tu máquina local, requieres ejecutar los siguientes comandos:

<pre>
apt-get install python3-pip
pip3 install pandas
pip3 install seaborn
pip3 install matplotlib
pip3 install pil
pip3 install wordcloud
pip3 install IPython
</pre>


Recuerda que mientras estés en ATGlabs, los comandos anteriores no son necesarios

### 1. Cargamos nuestras bibliotecas

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud
from wordcloud import STOPWORDS
from wordcloud import ImageColorGenerator
from IPython.display import HTML

### 2. Definimos el nombre de la muestra que nos tocó procesar

In [ ]:
sample_name = "S1"

### 3. Leemos nuestro archivo VCF con la función `read_csv()` de pandas y obtenemos un elemento al que llamaremos *dataframe*

In [ ]:
vcf_file = sample_name + "_snpeff_dbsnp_clinvar.vcf"
df = pd.read_csv(vcf_file,
                 sep="\t",
                 comment="#",
                 dtype="str",
                 header=None,
                 names=["chr","pos","id","ref","alt","qual","filter","info","format",sample_name])

#### El campo 8 de nuestro VCF está lleno de información que si bien es útil, no podemos procesarla toda de forma simultanea y tampoco podemos hacer sentido de esa información sin transformarla a algo más amigable

#### Tomemos por ejemplo la primera fila de nuestro VCF

In [ ]:
df["info"][0]

### 4. Creamos una función para transformar la estructura del campo 8 del VCF a un `diccionario` de python

In [ ]:
def get_var_info(info_str):
    info_list = info_str.split(";")
    info_dict = {}
    for element in info_list:
        if("=" in element):
            element_key   = element.split("=")[0]
            element_value = element.split("=")[1]
            info_dict[element_key] = element_value
    return info_dict

#### 4.1. Y transformamos nuestro campo 8 en algo menos agresivo

In [ ]:
vcf_df = df.copy()
vcf_df["info"] = vcf_df["info"].apply(lambda x: get_var_info(x))

In [ ]:
vcf_df["info"][0]

### 5. Creamos una función para obtener el campo "COMMON" de nuestras variantes

In [ ]:
def get_common_stat(info_dict):
    if("COMMON" in info_dict):
        common_stat = info_dict["COMMON"].split(",")
    else:
        common_stat = False
    return common_stat

#### 5.1. Y agregamos esa columna a nuestro dataframe

In [ ]:
vcf_df["common"] = vcf_df["info"].apply(lambda x: get_common_stat(x))

### 6. Creamos una función para obtener el significado clínico de nuestras variantes

In [ ]:
def get_clnsig_list(info_dict):
    if("CLNSIG" in info_dict):
        clnsig_list = info_dict["CLNSIG"].split(",")
    else:
        clnsig_list = False
    return clnsig_list

#### 6.1. Y agregamos esa columna a nuestro dataframe

In [ ]:
vcf_df["clnsig"] = vcf_df["info"].apply(lambda x: get_clnsig_list(x))

### 7. Creamos una función para obtener los fenotipos asociados a nuestras variantes

In [ ]:
def get_clndn_list(info_dict):
    if("CLNDN" in info_dict):
        clndn_list = info_dict["CLNDN"].split("|")
    else:
        clndn_list = False
    return clndn_list

#### 7.1. Y agregamos esa columna a nuestro dataframe

In [ ]:
vcf_df["clndn"] = vcf_df["info"].apply(lambda x: get_clndn_list(x))

### 8. Creamos una función adicional para obtener el campo "ANN" de nuestras variantes

In [ ]:
def get_ann_list(info_dict):
    if("ANN" in info_dict):
        ann_list = info_dict["ANN"].split(",")
    else:
        ann_list = False
    return ann_list

#### 8.1. Y agregamos esa columna a nuestro dataframe

In [ ]:
vcf_df["ann"] = vcf_df["info"].apply(lambda x: get_ann_list(x))

### 9. Creamos una función adicional para agregar links a ensembl

In [ ]:
def get_ensembl_link(rs_str):
    if "rs" in rs_str:
        url_str = "<a href=\"https://www.ensembl.org/homo_sapiens/Variation/Summary?v=" + rs_str + "\" target=\"_blank\">"+rs_str+"</a>"
    elif "." in rs_str:
        url_str = "."
    else:
        url_str = False
    return url_str

#### 9.1. Y agregamos esa columna a nuestro dataframe

In [ ]:
vcf_df["id"] = vcf_df["id"].apply(lambda x: get_ensembl_link(x))

### 10. Limpiamos nuestro dataframe y nos quedamos con solo algunas columnas

In [ ]:
vcf_df = vcf_df[["chr","pos","id","ref","alt","common","ann","clnsig","clndn",sample_name]]

#### 10.1. La columna de anotaciones sigue teniendo un output algo agresivo

#### Tomemos por ejemplo la primera fila de nuestro VCF

In [ ]:
vcf_df["ann"][0]

### 11. Convertimos las anotaciones del campo `ann` a un formato más amigable

In [ ]:
vcf_df = vcf_df.explode("ann")

#### 11.1. Construimos un nuevo *dataframe* y agregamos más columnas a dicho elemento a partir de la información de las anotaciones

In [ ]:
ann_names = {0:"allele",1:"effect",2:"impact",3:"gene_name",
             4:"gene_id",5:"feature_type",6:"feature_id",7:"feature_biotype",
             8:"rank",9:"hgvs_coding",10:"hgvs_protein",11:"cdna_pos",
             12:"cds_pos",13:"prot_pos",14:"distance",15:"messages"}
ann_df = vcf_df.copy()
ann_df = ann_df["ann"].str.split("|",expand=True).rename(ann_names,axis=1)

#### 11.2. Examinamos rapidamente el resultado

In [ ]:
ann_df.head()

### 12. Convertimos las anotaciones del campo `sample_name` a un formato más amigable

#### 12.1. Para ello, construimos un nuevo *dataframe* y agregamos más columnas a dicho elemento a partir de la información del genotipo, profundidades, calidades y likelihoods

In [ ]:
gt_names ={0:"genotype",1:"allele_depth",2:"depth_coverage",3:"genotype_quality",4:"phred_likelihood"}
gt_df = vcf_df.copy()
gt_df = gt_df[sample_name].str.split(":",expand=True).rename(gt_names,axis=1)

### 13. Integramos todo en un sólo *dataframe*

In [ ]:
clean_df = pd.concat([vcf_df.drop(["ann",sample_name],axis=1),ann_df,gt_df],axis=1)
clean_df["pos"] = clean_df["pos"].astype(int)

#### 13.1. Examinamos rapidamente el resultado

In [ ]:
HTML(clean_df.head().to_html(escape=False))

### 14. Y entonces podemos filtrar por genes, genotipos, cromosomas, etc.

#### 14.1. Distribución de genotipos a lo largo de los cromosomas

In [ ]:
plt.figure(figsize=(18,6))
ax = sns.stripplot(data=clean_df, x="chr", y="pos", hue="genotype")
ax.set(ylabel="")
plt.show()
plt.close()

In [ ]:
hom_gt = ["1/1"]
hom_df = clean_df.copy()
hom_df = hom_df[hom_df["genotype"].isin(hom_gt)]
sns.histplot(hom_df["chr"]);

In [ ]:
het_gt = ["0/1","1/2"]
het_df = clean_df.copy()
het_df = het_df[het_df["genotype"].isin(het_gt)]
sns.histplot(het_df["chr"]);

#### 14.2. Genes sobrerepresentados en el set de variantes

In [ ]:
gene_string = clean_df["gene_name"].to_list()
gene_string = str(gene_string).replace("\n","").replace(",","").replace("'","").strip()
wordcloud = WordCloud(background_color="white",width=1000, height=500).generate(gene_string)
plt.figure(figsize=(20,10))
plt.imshow(wordcloud);

#### 14.3. Variantes que pertenecen unicamente a un conjunto de genes

In [ ]:
gene_list = ["ABCA1","ABCB4","AC053503.6","ADAMTS13","ALMS1","ASCL1","ATP7B","BCHE","CDKN1B","CEP290","COL4A4","COL6A3","DES","EIF3J","EOMES","F13A1","F13B","FBN1","FGA","FTH1P1","GALC","GALNS","HCN4","IFNGR1","IL21R","KCNV2","KLKB1","NPC1","PAH","PCDH15","PCSK9","PLOD1","PPP4C","PRNP","RELN","SCN5A","SLC34A3","SNIP1","SPATA7","SPG11","STOX1","SUMO4","TAB2","TBX6","TTC8","TYR","VPS13B","ZFYVE26","ZFYVE27"]
panel_df  = clean_df.copy()
panel_df  = panel_df[panel_df["gene_name"].isin(gene_list)]
HTML(panel_df[["chr","pos","id","ref","alt","genotype","clnsig","clndn"]].to_html(escape=False))

#### 14.4. Y su interpretación clínica

In [ ]:
clnsig_string = panel_df["clnsig"].to_list()
clnsig_string = str(clnsig_string).replace("\n","").replace(",","").replace("'","").strip()
wordcloud = WordCloud(background_color="white",width=1000, height=500).generate(clnsig_string)
plt.figure(figsize=(20,10))
plt.imshow(wordcloud);